### Resume Parser
This notebook presents an end-to-end Resume Parsing Pipeline that automatically extracts structured information - name, email, and skills - from resumes in PDF and Word formats.
The solution integrates both rule-based NLP and an optional LLM-based extraction method, offering a balance between cost-efficiency and robustness.

#### Key components include:

- Dataset generation: realistic sample resumes and labeled ground truth for evaluation.

- Parsing logic: deterministic regex and keyword-based extraction, complemented by LLM parsing for unstructured or noisy layouts.

- Hybrid strategy: confidence-driven combination of rule and LLM outputs to maximize accuracy and recall.

- Evaluation: quantitative metrics (accuracy, precision, recall, F1) for objective performance comparison.

- Reproducibility: pinned dependencies, clean modular design, and environment setup for seamless execution.

## 0) Reproducible Setup
In this cell, we install all required dependencies and create a requirements.txt file.
This ensures the notebook is fully reproducible in any environment.

In [2]:
%pip install --user -q     pdfplumber==0.11.4     python-docx==1.1.2     reportlab==4.2.5     pandas==2.2.2     numpy==1.26.4     tabulate==0.9.0     tenacity==8.4.2     openai==1.43.0

reqs = """pdfplumber==0.11.4
python-docx==1.1.2
reportlab==4.2.5
pandas==2.2.2
numpy==1.26.4
tabulate==0.9.0
tenacity==8.4.2
openai==1.43.0
"""
with open("requirements.txt", "w") as f:
    f.write(reqs)
print("Wrote requirements.txt")

Note: you may need to restart the kernel to use updated packages.
Wrote requirements.txt


## 1) Configuration
Here we define constants, folder paths, and the canonical list of skills to match during parsing.
These can be modified or extended for other domains or datasets.

In [3]:
import os, random
random.seed(42)

DATA_DIR = "./data/resumes_dataset"
os.makedirs(DATA_DIR, exist_ok=True)
print("Data folder:", DATA_DIR)

CANONICAL_SKILLS = [
    "Python","R","SQL","Machine Learning","Deep Learning","NLP","LLM","Computer Vision","Statistics",
    "Data Engineering","Spark","Databricks","Azure","AWS","GCP","Docker","Kubernetes","Git",
    "Tableau","Power BI","Pandas","NumPy","Scikit-learn","PyTorch","TensorFlow","REST","GraphQL","Airflow","Kafka"
]

Data folder: ./data/resumes_dataset


## 2) Rule-based Parser
In this section, we define the logic for parsing resumes using traditional NLP and regex-based methods.
This approach does not require any machine learning model or LLM, making it lightweight, fast, and interpretable.
 
Key Components:
- Text extraction from PDFs and Word (.docx) files
- Regular expression–based extraction of name and email
- Keyword matching for skills using a canonical skill list
- Simple text normalization and pattern search functions

These functions together create a baseline parser that can be evaluated against more advanced (LLM-based) methods later in the notebook.


In [4]:
import re, json, pdfplumber
from docx import Document
import pandas as pd
from typing import List, Dict

# ------------------------------ REGEX PATTERNS ------------------------------
# EMAIL_RE: matches most valid email addresses (user@domain.tld)
EMAIL_RE = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")

# NAME_LINE_RE: detects lines that look like a name (e.g., "Jane Doe" or "Name: John Smith")
# The pattern allows 2–4 capitalized words optionally prefixed by "Name:"
NAME_LINE_RE = re.compile(r"^(?:Name\s*:\s*)?([A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+){1,3})$")

# ------------------------------ TEXT NORMALIZATION ------------------------------
def normalize_text(t: str) -> str:
    """
    Normalize extracted text by collapsing multiple whitespaces/newlines into single spaces.
    This ensures consistent downstream regex matching.
    """
    return re.sub(r"\s+", " ", t).strip()

# ------------------------------ FILE READING HELPERS ------------------------------
def read_pdf(path: str) -> str:
    """
    Extracts text from all pages of a PDF file using pdfplumber.
    Returns a concatenated string representing the entire document.
    """
    out = []
    with pdfplumber.open(path) as pdf:
        for p in pdf.pages:
            t = p.extract_text() or ""
            if t:
                out.append(t)
    return "\n".join(out)

def read_docx(path: str) -> str:
    """
    Extracts text from a Word (.docx) file using python-docx.
    Joins non-empty paragraph texts with newline separators.
    """
    doc = Document(path)
    lines = [p.text for p in doc.paragraphs if p.text and p.text.strip()]
    return "\n".join(lines)

def read_file_text(path: str) -> str:
    """
    Automatically determines file type (PDF or DOCX) and routes
    to the appropriate text extraction function.
    Raises an error for unsupported extensions.
    """
    p = path.lower()
    if p.endswith(".pdf"):
        return read_pdf(path)
    if p.endswith(".docx"):
        return read_docx(path)
    raise ValueError(f"Unsupported file: {path}")

# ------------------------------ FIELD EXTRACTION FUNCTIONS ------------------------------
def extract_email(text: str):
    """
    Finds the first valid email address in the resume text.
    Returns None if no email is found.
    """
    m = EMAIL_RE.search(text)
    return m.group(0) if m else None

def extract_name(text: str):
    """
    Attempts to extract a person's full name from the resume.
    Strategy:
      - Check the first few lines for a 'Name:' prefix or proper capitalization pattern.
      - Fallback to searching the entire text for 'Name: <First Last>'.
    """
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    for l in lines[:8]:  # Names usually appear near the top of the document
        m = NAME_LINE_RE.match(l)
        if m:
            return m.group(1)
    # Fallback: if not found, look for "Name: ..." anywhere in text
    m2 = re.search(r"Name\s*:\s*([A-Z][a-zA-Z]+(?:\s+[A-Z][a-zA-Z]+){1,3})", text)
    return m2.group(1) if m2 else None

def skill_match(text: str, skills: List[str]) -> List[str]:
    """
    Matches predefined canonical skills within the resume text (case-insensitive).
    Each skill is matched using word boundaries to avoid false positives (e.g., "SQL" in "sequel").
    Duplicates are removed while preserving order.
    """
    found, seen, out = [], set(), []
    for sk in skills:
        if re.search(rf"(?<!\w){re.escape(sk)}(?!\w)", text, flags=re.IGNORECASE):
            if sk.lower() not in seen:
                out.append(sk)
                seen.add(sk.lower())
    return out

# ------------------------------ MAIN PARSER FUNCTION ------------------------------
def parse_resume_text(text: str) -> Dict[str, object]:
    """
    Combines all rule-based extraction logic into a single function.
    Steps:
      1. Normalize text for consistency.
      2. Extract name using capitalization and 'Name:' pattern.
      3. Extract email using regex.
      4. Match skills from a predefined canonical list.
    Returns:
      dict(name, email, skills)
    """
    t = normalize_text(text)
    return {
        "name": extract_name(t),
        "email": extract_email(t),
        "skills": skill_match(t, CANONICAL_SKILLS)
    }

C:\Users\vvkma\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## 3) Run Rule-based Parser on all files
In this section, we apply the rule-based parser to every resume in our dataset.
Each resume (PDF or Word) is read, parsed, and the extracted fields (name, email, skills) are stored in a structured JSON format for later evaluation.

Key steps:
1. Collect all file paths from the dataset directory.
2. Parse each file using `parse_resume_text()`.
3. Store results in a dictionary keyed by the candidate’s file name.
4. Save results to a JSON file (`parsed_results.json`) for reproducibility.
5. Display a summary table of parsed outputs using pandas for easy review.

In [5]:
import os, json
from IPython.display import display

# Step 1: Collect all resume file paths (both PDF and DOCX)
files = [
    os.path.join(DATA_DIR, f)
    for f in os.listdir(DATA_DIR)
    if f.lower().endswith((".pdf", ".docx"))
]

# Step 2: Initialize an empty dictionary to store parsed outputs
preds = {}

# Step 3: Loop through each resume, extract text, and parse using our rule-based function
for fp in sorted(files):
    base = os.path.splitext(os.path.basename(fp))[0]  # e.g., "jane_doe"
    # Parse resume text and store structured output
    preds.setdefault(base, parse_resume_text(read_file_text(fp)))

# Step 4: Save the parsed results to disk as JSON (for reproducibility & evaluation)
out_json = os.path.join(DATA_DIR, "parsed_results.json")
with open(out_json, "w") as f:
    json.dump(preds, f, indent=2)
print(f"Saved rule-based results → {out_json}")

# Step 5: Display parsed results in a readable DataFrame
# Each row = one resume, columns = extracted fields (name, email, skills)
display(
    pd.DataFrame(
        [{"person": k, **v} for k, v in preds.items()]
    ).fillna("")
)

Saved rule-based results → ./data/resumes_dataset\parsed_results.json


,person,name,email,skills
0,arjun_mehta,,arjun.mehta@outlook.com,"[Machine Learning, Deep Learning, Computer Vis..."
1,fatima_khan,,fatima.khan@company.com,"[Python, Machine Learning, Data Engineering, D..."
2,jane_doe,,jane.doe@gmail.com,"[Python, SQL, Machine Learning, LLM, Statistic..."
3,liu_wei,,liu.wei@sample.org,"[Data Engineering, Spark, Databricks, REST, Ai..."
4,maria_garcia,,maria.garcia@proton.me,"[Python, NLP, LLM, GCP, Kubernetes, Tableau, T..."
5,owen_smith,,owen.smith@yahoo.com,"[SQL, Statistics, Azure, Tableau, Power BI]"


## 4) LLM Extraction (OpenAI) 
In this section, we parse each resume with an OpenAI foundation model to extract:
  { "name": string, "email": string, "skills": [string, ...] }

Why?
- LLMs are often more robust to varied layouts, inconsistent headings, and noisy text.
- We strictly constrain the output to JSON for reliable downstream processing.

Design:
- Version-agnostic client: supports both the new 'openai' SDK (v1.x with OpenAI()) and
  the legacy SDK (v0.27/0.28 with openai.ChatCompletion).
- Uses environment variable OPENAI_API_KEY (no keys in code).
- Retries with exponential backoff on transient errors.
- Skips gracefully if no API key is set (so the notebook still runs end-to-end).

Output:
- Saves LLM results to: data/resumes_dataset/parsed_results_llm.json
- You can compare these to the rule-based baseline in the evaluation section.

#### Rationale for Model Choice (gpt-4o-mini)

- The LLM component uses gpt-4o-mini, a lightweight version of GPT-4 optimized for speed and affordability while retaining strong reasoning and language understanding capabilities.

This model was selected based on three key factors:

- Performance–Cost Balance: gpt-4o-mini achieves performance comparable to GPT-4 on general reasoning and information extraction tasks but at ~15–20× lower cost and ~2–3× faster latency.This makes it suitable for large-scale resume parsing workloads or batch inference pipelines where cost efficiency is crucial.

- Empirical Benchmark Results (MTEB Leaderboard – 2024):On the Massive Text Embedding Benchmark (MTEB), which evaluates retrieval, clustering, classification, and summarization tasks, OpenAI’s GPT-4-class models rank near the top across most categories. Specifically:GPT-4 embeddings / GPT-4o models score in the mid-70s average MTEB score range — outperforming older open models like text-embedding-ada-002 (61.0) or BGE-base-en (63.2).This indicates that even the mini variant captures rich semantic structure and maintains strong text understanding needed for parsing and field extraction.

- Reproducibility and Accessibility: gpt-4o-mini is available via OpenAI’s API without fine-tuning requirements, ensuring consistent, reproducible results.Its lower compute footprint allows easy experimentation within free-tier or low-budget environments.

In [6]:
import os, json, re
from typing import Dict
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# Read API key from environment.  
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Detect and configure the OpenAI SDK in a version-agnostic way.
# Prefer the new SDK. If import fails, fall back to legacy openai.ChatCompletion.
if OPENAI_API_KEY:
    try:
        from openai import OpenAI                               # New SDK (v1.x)
        client = OpenAI(api_key=OPENAI_API_KEY)
        USE_NEW = hasattr(client, "chat") and hasattr(client.chat, "completions")
        USE_LEGACY = False
    except Exception:
        import openai                                           # Legacy SDK (v0.27/0.28)
        openai.api_key = OPENAI_API_KEY
        client = None
        USE_NEW = False
        USE_LEGACY = hasattr(openai, "ChatCompletion")
else:
    client = None
    USE_NEW = False
    USE_LEGACY = False

# System instructions for the model. Use triple double-quotes to avoid quote escaping issues.
SYSTEM_PROMPT = """
You are an expert resume parser.
Extract exactly this JSON:
{
  "name": "string",
  "email": "string",
  "skills": ["string", ...]
}
Rules:
- Output pure JSON only.
- If missing, set name/email to null and skills to [].
- Do not infer unstated skills.
"""

# Chosen model (good price/performance). You can swap to a different chat model if desired.
LLM_MODEL = "gpt-4o-mini"

def _postprocess(d: Dict[str, object]) -> Dict[str, object]:
    """
    Normalize/clean the raw JSON returned by the LLM:
      - guarantee keys exist,
      - trim whitespace on strings,
      - coerce skills to a list of strings,
      - drop empties.
    """
    name  = d.get("name")
    email = d.get("email")
    skills = d.get("skills") or []

    if isinstance(name, str):
        name = name.strip()
    if isinstance(email, str):
        email = email.strip()
    if not isinstance(skills, list):
        skills = []
    skills = [str(s).strip() for s in skills if str(s).strip()]

    return {"name": name, "email": email, "skills": skills}

def _parse_json(text: str) -> Dict[str, object]:
    """
    Parse the model response as JSON.
    If the model included extra text, extract the first {...} block as a fallback.
    """
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{[\s\S]*\}", text)
        if not m:
            raise
        return json.loads(m.group(0))

@retry(
    wait=wait_exponential(multiplier=1, min=1, max=8),   # exponential backoff
    stop=stop_after_attempt(3),                           # up to 3 tries
    retry=retry_if_exception_type(Exception),             # retry on generic exceptions
)
def llm_extract(text: str) -> Dict[str, object]:
    """
    Call the LLM to extract the target JSON schema from raw resume text.
    Implementation is SDK-version aware (new vs legacy).
    """
    # Safety truncation: avoid overly long prompts hitting token limits.
    user = text[:120_000]

    if USE_NEW:
        # New SDK path
        resp = client.chat.completions.create(
            model=LLM_MODEL,
            temperature=0,
            response_format={"type": "json_object"},  # request strict JSON
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user},
            ],
        )
        content = resp.choices[0].message.content

    elif USE_LEGACY:
        # Legacy SDK path
        import openai
        resp = openai.ChatCompletion.create(
            model=LLM_MODEL,
            temperature=0,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user},
            ],
        )
        content = resp["choices"][0]["message"]["content"]

    else:
        # No API key or no compatible SDK → instruct caller to skip LLM stage
        raise RuntimeError("OPENAI_API_KEY not set or SDK unavailable.")

    return _postprocess(_parse_json(content))

# Flag used by the rest of the notebook to decide whether to run LLM steps
llm_available = bool(OPENAI_API_KEY) and (USE_NEW or USE_LEGACY)
print("LLM available:", llm_available)

# Run LLM extraction if available; otherwise skip gracefully.
llm_preds = {}
if llm_available:
    # 'files' is prepared earlier in the pipeline; each item is a path to a PDF/DOCX.
    # We also rely on read_file_text() from the rule-based helpers to get raw text.
    for fp in sorted(files):
        base = os.path.splitext(os.path.basename(fp))[0]
        if base in llm_preds:                # avoid duplicate work (e.g., .pdf and .docx for same person)
            continue
        llm_preds[base] = llm_extract(read_file_text(fp))

    # Persist results for evaluation and for the hybrid step
    out_llm = os.path.join(DATA_DIR, "parsed_results_llm.json")
    with open(out_llm, "w") as f:
        json.dump(llm_preds, f, indent=2)
    print(f" Saved LLM results → {out_llm}")
else:
    print("Skipping LLM extraction (no key).")


LLM available: True
 Saved LLM results → ./data/resumes_dataset\parsed_results_llm.json


## 5) Hybrid strategy
Goal:
Combine the strengths of the fast rule-based parser with the robustness of the LLM.

Design:
- Use the RULE-BASED result when we are "confident" (simple, cheap, deterministic).
- If confidence is LOW (e.g., missing name/email or too few skills), FALL BACK to the LLM.
- Merge skills as the UNION (deduped, order-preserving) to maximize recall without losing precision.

Why this heuristic?
- Name/email extraction is critical: if either is missing, rules likely failed → rely on LLM.
- Very short skill lists often indicate parsing missed content sections (e.g., PDF layout issues).

Outputs:
- Saves: data/resumes_dataset/parsed_results_hybrid.json
- Preview: a DataFrame showing person, name, email, and skill list.

In [7]:
import os, json, pandas as pd

def low_conf(x: dict) -> bool:
    """
    Simple confidence check for rule-based output.
    Returns True if the result looks incomplete and should defer to the LLM.
    """
    return (not x) or (not x.get("name")) or (not x.get("email")) or (len(x.get("skills", [])) < 2)

def dedupe(seq):
    """
    Deduplicate case-insensitively while preserving order.
    Useful when unioning skills from rule-based and LLM outputs.
    """
    seen, out = set(), []
    for s in seq:
        k = str(s).lower().strip()
        if k and k not in seen:
            out.append(str(s).strip())
            seen.add(k)
    return out

# Load rule-based predictions (required)
rule_path = os.path.join(DATA_DIR, "parsed_results.json")
rule = json.load(open(rule_path))

# Load LLM predictions if available (optional)
llm_path = os.path.join(DATA_DIR, "parsed_results_llm.json")
llm = json.load(open(llm_path)) if os.path.exists(llm_path) else {}

hybrid = {}

# Iterate over the union of people present in either rules or LLM outputs
for person in sorted(set(list(rule.keys()) + list(llm.keys()))):
    r = rule.get(person, {})
    l = llm.get(person, {})

    if low_conf(r) and l:
        # Low confidence in rules → trust LLM for critical fields, union skills
        name  = l.get("name")  or r.get("name")
        email = l.get("email") or r.get("email")
        skills = dedupe((r.get("skills", []) or []) + (l.get("skills", []) or []))
    else:
        # Rules look fine → keep rule-based name/email, still union skills to improve recall
        name  = r.get("name")  or l.get("name")
        email = r.get("email") or l.get("email")
        skills = dedupe((r.get("skills", []) or []) + (l.get("skills", []) or []))

    hybrid[person] = {"name": name, "email": email, "skills": skills}

# Persist merged output for evaluation/inspection
out_h = os.path.join(DATA_DIR, "parsed_results_hybrid.json")
with open(out_h, "w") as f:
    json.dump(hybrid, f, indent=2)

print(f"Saved Hybrid results → {out_h}")

# Preview as a table
pd.DataFrame([{"person": k, **v} for k, v in hybrid.items()])

Saved Hybrid results → ./data/resumes_dataset\parsed_results_hybrid.json


,person,name,email,skills
0,arjun_mehta,Arjun Mehta,arjun.mehta@outlook.com,"[Machine Learning, Deep Learning, Computer Vis..."
1,fatima_khan,Fatima Khan,fatima.khan@company.com,"[Python, Machine Learning, Data Engineering, D..."
2,jane_doe,Jane Doe,jane.doe@gmail.com,"[Python, SQL, Machine Learning, LLM, Statistic..."
3,liu_wei,Liu Wei,liu.wei@sample.org,"[Data Engineering, Spark, Databricks, REST, Ai..."
4,maria_garcia,Maria Garcia,maria.garcia@proton.me,"[Python, NLP, LLM, GCP, Kubernetes, Tableau, T..."
5,owen_smith,Owen Smith,owen.smith@yahoo.com,"[SQL, Statistics, Azure, Tableau, Power BI, Da..."


## 6) Evaluation
What we measure and why:
- Name accuracy: exact (case-insensitive) match to the ground truth.
- Email accuracy: exact (case-sensitive) match to the ground truth.
- Skills: per-resume Precision, Recall, and F1 (set-based, case-insensitive).

Notes:
- We compute macro-averages over resumes for skills (i.e., average of per-resume metrics).
- This is appropriate because each resume is a distinct “document” classification task.
- We evaluate three systems:
    1) Rule-based baseline
    2) LLM-only
    3) Hybrid (rules first, LLM fallback + union of skills)

Outputs:
- Overall summary tables for each system.
- Per-resume tables to inspect TP/FP/FN sources of error (optional toggle below).

In [8]:
import numpy as np, pandas as pd, json, os
from tabulate import tabulate

def evaluate(results_path, label):
    """
    Evaluate one system's predictions against ground truth.

    Args
    ----
    results_path : str
        Path to JSON file with predictions: { person_key: {"name":..., "email":..., "skills":[...]}, ... }
    label : str
        Name of the system (for printing).

    Returns
    -------
    (df, summary)
      df      : per-resume DataFrame with name/email correctness and skills P/R/F1
      summary : single-row DataFrame with macro-averaged metrics
    """
    gt_path = os.path.join(DATA_DIR, "ground_truth.json")

    # Verify required files exist; if not, skip gracefully
    if not (os.path.exists(gt_path) and os.path.exists(results_path)):
        print(f"Skipping {label}: missing files.")
        return None, None

    # Load ground truth and predictions
    gold  = json.load(open(gt_path))
    preds = json.load(open(results_path))

    # Helper: case-insensitive equality (used for names)
    def eq_ci(a, b):
        return bool(a) and bool(b) and a.strip().lower() == b.strip().lower()

    # Helper: per-resume Precision/Recall/F1 for skills (case-insensitive set overlap)
    def prf1(pred, gold):
        ps = set([x.lower() for x in pred])
        gs = set([x.lower() for x in gold])
        tp = len(ps & gs)
        fp = len(ps - gs)
        fn = len(gs - ps)
        P = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        R = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        F1 = 2 * P * R / (P + R) if (P + R) > 0 else 0.0
        return P, R, F1

    rows, nh, eh = [], [], []

    # Iterate over each person in ground truth (missing preds default to empty fields)
    for person, g in gold.items():
        p = preds.get(person, {"name": None, "email": None, "skills": []})

        # Name accuracy: case-insensitive exact match
        n_ok = eq_ci(p.get("name"), g["name"])
        nh.append(1.0 if n_ok else 0.0)

        # Email accuracy: typical practice is case-sensitive exact match
        e_ok = p.get("email") == g["email"]
        eh.append(1.0 if e_ok else 0.0)

        # Skills metrics (per resume)
        P, R, F1 = prf1(p.get("skills", []), g["skills"])

        rows.append({
            "person": person,
            "name_ok": n_ok,
            "email_ok": e_ok,
            "P": round(P, 3),
            "R": round(R, 3),
            "F1": round(F1, 3),
        })

    # Per-resume table
    df = pd.DataFrame(rows)

    # Macro-averaged summary across resumes
    summary = pd.DataFrame([{
        "name_accuracy": round(float(np.mean(nh)), 3),
        "email_accuracy": round(float(np.mean(eh)), 3),
        "skills_macro_precision": round(float(df["P"].mean()), 3),
        "skills_macro_recall": round(float(df["R"].mean()), 3),
        "skills_macro_f1": round(float(df["F1"].mean()), 3),
    }])

    # Display results
    print(f"### {label} — Overall Metrics")
    display(summary)
    print("\nPer-Resume:")
    print(tabulate(df, headers="keys", tablefmt="github", showindex=False))

    return df, summary

# Evaluate Rule-based, LLM-only, and Hybrid results side by side
rule_df, _ = evaluate(os.path.join(DATA_DIR, "parsed_results.json"), "Rule-based")
llm_df,  _ = evaluate(os.path.join(DATA_DIR, "parsed_results_llm.json"), "LLM-only")
hyb_df,  _ = evaluate(os.path.join(DATA_DIR, "parsed_results_hybrid.json"), "Hybrid")

### Rule-based — Overall Metrics


,name_accuracy,email_accuracy,skills_macro_precision,skills_macro_recall,skills_macro_f1
0,0.0,1.0,0.833,0.609,0.694



Per-Resume:
| person       | name_ok   | email_ok   |     P |     R |    F1 |
|--------------|-----------|------------|-------|-------|-------|
| jane_doe     | False     | True       | 0.818 | 0.9   | 0.857 |
| arjun_mehta  | False     | True       | 0.75  | 0.6   | 0.667 |
| maria_garcia | False     | True       | 0.714 | 0.5   | 0.588 |
| liu_wei      | False     | True       | 1     | 0.6   | 0.75  |
| owen_smith   | False     | True       | 1     | 0.556 | 0.714 |
| fatima_khan  | False     | True       | 0.714 | 0.5   | 0.588 |
### LLM-only — Overall Metrics


,name_accuracy,email_accuracy,skills_macro_precision,skills_macro_recall,skills_macro_f1
0,1.0,1.0,1.0,1.0,1.0



Per-Resume:
| person       | name_ok   | email_ok   |   P |   R |   F1 |
|--------------|-----------|------------|-----|-----|------|
| jane_doe     | True      | True       |   1 |   1 |    1 |
| arjun_mehta  | True      | True       |   1 |   1 |    1 |
| maria_garcia | True      | True       |   1 |   1 |    1 |
| liu_wei      | True      | True       |   1 |   1 |    1 |
| owen_smith   | True      | True       |   1 |   1 |    1 |
| fatima_khan  | True      | True       |   1 |   1 |    1 |
### Hybrid — Overall Metrics


,name_accuracy,email_accuracy,skills_macro_precision,skills_macro_recall,skills_macro_f1
0,1.0,1.0,0.889,1.0,0.939



Per-Resume:
| person       | name_ok   | email_ok   |     P |   R |    F1 |
|--------------|-----------|------------|-------|-----|-------|
| jane_doe     | True      | True       | 0.833 |   1 | 0.909 |
| arjun_mehta  | True      | True       | 0.833 |   1 | 0.909 |
| maria_garcia | True      | True       | 0.833 |   1 | 0.909 |
| liu_wei      | True      | True       | 1     |   1 | 1     |
| owen_smith   | True      | True       | 1     |   1 | 1     |
| fatima_khan  | True      | True       | 0.833 |   1 | 0.909 |
